In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score

nltk.download('stopwords')
from nltk.corpus import stopwords


label_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}


music_recommendations = {
    'sadness': ['Melancholic Piano', 'Sad Violin Music'],
    'joy': ['Happy Acoustic Guitar', 'Uplifting Piano'],
    'love': ['Romantic Piano', 'Love Songs Instrumental'],
    'anger': ['Intense Rock Instrumental', 'Heavy Metal Instrumental'],
    'fear': ['Dark Cinematic Music', 'Tense Ambient Soundscapes'],
    'surprise': ['Energetic Orchestral Music', 'Exciting Electronic Beats'],
}


data = pd.read_csv('emotions.csv')


stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    text = text.lower()


    contractions = {
        "dont": "do not",
        "cant": "cannot",
        "wont": "will not",
        "im": "i am",
        "ive": "i have",
        "id": "i would",
        "youre": "you are",
        "isnt": "is not",
        "wasnt": "was not",
        "shouldnt": "should not",
        "couldnt": "could not",
        "doesnt": "does not",
        "havent": "have not",
        "hasnt": "has not",
        "hadnt": "had not",
        "arent": "are not",
        "werent": "were not",
        "wouldnt": "would not",
        "mustnt": "must not",
        "mightnt": "might not",
        "didnt": "did not",
        "neednt": "need not",
        "oughtnt": "ought not",
        "im": "i am",
        "hes": "he is",
        "shes": "she is",
        "its": "it is",
        "thats": "that is",
        "theres": "there is",
        "whats": "what is",
        "wheres": "where is",
        "whos": "who is",
        "theyre": "they are",
        "weve": "we have",
        "were": "we are",
        "didnt": "did not",
        "doesnt": "does not",
        "dont": "do not",
        "hadnt": "had not",
        "hasnt": "has not",
        "havent": "have not",
        "isnt": "is not",
        "shouldnt": "should not",
        "wasnt": "was not",
        "werent": "were not",
        "wont": "will not",
        "wouldnt": "would not",
    }


    for contraction, replacement in contractions.items():
        text = re.sub(r'\b' + contraction + r'\b', replacement, text)


    tokens = text.split()
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    tokens = [token for token in tokens if token and token not in stop_words]

    return tokens


data['processed_text'] = data['text'].apply(preprocess_text)


X = data['processed_text']
y = data['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4, sg=1)


def get_sentence_vector(tokens, model, vector_size=100):
    if not tokens:
        return np.zeros(vector_size)
    valid_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if not valid_vectors:
        return np.zeros(vector_size)
    return np.mean(valid_vectors, axis=0)


X_train_vectors = np.array([get_sentence_vector(tokens, w2v_model) for tokens in X_train])
X_test_vectors = np.array([get_sentence_vector(tokens, w2v_model) for tokens in X_test])


from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train_vectors, y_train)

y_pred = classifier.predict(X_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy:.4f}")


def predict_emotion(text):
    tokens = preprocess_text(text)
    text_vector = get_sentence_vector(tokens, w2v_model)
    predicted_label = classifier.predict([text_vector])[0]
    predicted_emotion = label_mapping[predicted_label]
    return predicted_emotion


user_input = input("Enter your input: ")
detected_emotion = predict_emotion(user_input)
print(f"Detected Emotion: {detected_emotion}")
print(f"Recommended Music for {detected_emotion}: {music_recommendations.get(detected_emotion, [])}")
